#  Задание 1

1. Какие гипотезы называются простыми, а какие сложными?

    * Простая гипотеза - тестируется равенство наблюдаемой выборки одному распределению. Альтернатива этому - сложная гипотеза

2. Чем отличаются параметрические критерии от непараметрических? В каких ситуациях можно использовать параметрические критерии, а в каких лучше использовать непараметрические?

    * Параметрические методы - мы предполагаем, что выборка взята из параметрического распределения и в процессе тестирования оцениваем или используем параметры этого распределения. Для непараметрических критериев мы не делаем таких предположений.
    * Если есть возможность использовать параметрические критерии - доверительные интервалы по ним могут оказаться уже и помочь нам протестировать гипотезу с меньшим числом наблюдений на том же уровне $pvalue$. Когда у нас данные категариальные или мы не знаем, к какому распределению принадлежит выборка, то может быть необходимость использовать непараметрические критерии. Также, часть непараметрических критериев проще в реализации.

3. Что такое уровень значимости? А фактический уровень значимости?

    * Уровень значимости - вероятность отвергнуть нулевую гипотезу, при условии что она верна. Для доверительного интервала - если $\alpha$ = равно уровень значимости, то доверительный интервал, который мы строим на этом уровне значимости будет накрывать значение, для которого мы строим доверительный интервал с вероятностью $1-\alpha$
    * фактический уровень значимости - pvalue. Это вероятность для верной нулевой гипотезы $H0$ получить
значение статистики равное или более экстремальное, тому, что мы посчитали по выборке

4. С помощью каких критериев можно проверить нормальность выборки?

    * Визуально по графику $Q-Q$
    * Критерий Колмогорова-Смирнова, оценив параметры нормального распределения по выборке и сравнив с выборочной функцией распределения
    * Шапиро-Уилка
    * Харке-Бера
    * Хи-квадрат, развив данные на бакеты
